In [2]:
from random import random
import generators as gen
from math import pi, log, sin

#TIEMPOS DE ARRIBO
def lamda_t(t):
    return 30 + 30*sin((2*pi*t)/24)
    
def Poisson_NH(T):
    NT = 0
    Eventos = []
    lamda = 7
    t = -log(1 - random()) / lamda
    while t <= T:
        V = random()
        if V < lamda_t(t) / lamda:
            NT += 1
            Eventos.append(t)
        t += -log(1 - random()) / lamda
    return NT, Eventos

def exp(lamda):
    u =  random()
    return -log(u)/lamda

In [8]:
def queueSystem(hours=48):
    n, arribos =  Poisson_NH(hours)
    servicios = [exp(40) for _ in arribos]
    esperas = []
    uso = [[0,0]]
    cola = [[0,0]]

    arriboAnt = 0
    for i in range(n):
        inicio = max(arribos[i], arriboAnt)
        espera = inicio - arribos[i]
        esperas.append(espera)
        
        if espera != 0:
            uso[-1][1] += servicios[i]
            cola.append([arribos[i], cola[-1][1] + 1])
        else:
            uso.append([inicio, inicio + servicios[i]])
            cola.append([arribos[i], max(cola[-1][1] - 1, 0)])
                    
        arriboAnt = inicio + servicios[i]

    return arribos, servicios, esperas, uso, cola

queueSystem()

([0.17144171972780015,
  0.1795656960486837,
  0.4636123904102518,
  0.5420799693014492,
  0.6251509522316128,
  0.9783361053614428,
  1.0325546928975369,
  1.0652953128371216,
  1.0872830295281934,
  1.233210727724295,
  1.2572478127298614,
  1.300188560309985,
  1.3293262720963372,
  1.5083696953035397,
  1.5187016557132509,
  1.645181106366254,
  1.6719350330557967,
  2.2371684267695233,
  2.4354273590243833,
  3.0559179646312153,
  3.2074134490009887,
  3.694695564311406,
  3.835087379792227,
  4.170758810467289,
  4.399284696490532,
  4.905352462283755,
  5.144619610246977,
  5.253696132516012,
  5.271793899921216,
  5.299249667517119,
  5.555044504091271,
  5.611873848327248,
  5.6616655604322395,
  5.687762117812444,
  5.931047875408642,
  5.979714216853707,
  6.03285796386351,
  6.05591140368064,
  6.206569910671164,
  6.364015612085744,
  6.467533109660088,
  6.580953452192284,
  6.59062910373039,
  6.667725586110589,
  7.019966421070297,
  7.339171572834688,
  7.7323255615998

In [9]:
##Tasa de Utilizacion del servicio.
def tasa_servicio(usos):
    tasa = [0]*48
    tiempo_extra = 0
    for h in range(48):
        if tiempo_extra != 0:
            tasa[h] = tiempo_extra
            tiempro_extra = 0
        for uso in usos:
            if uso[0] > h:
                if uso[1] < h + 1:  #Uso en [h, h+1)
                    tasa[h] += uso[1]-uso[0]
                elif uso[0] < h + 1:  #Uso en [h, h+2), sumamos solo la parte en [h, h+1) y el extra lo guardamos para la proxima hora             
                    tiempo_extra = uso[1] - (h + 1)
                    tasa[h] += ((h + 1) - uso[0])
                else: #No hay mas usos en [h, h+1], y pasamos a la proxima hora
                    break
    return tasa

#Porcentaje de tiempo del servido ocupado
def uso_total_servicio(tasa):
    total = 0
    for t in tasa:
        total += t
    return total / 48

def promedio_tasa(Nsim):
    tasa_est = [0]*48
    for _ in range(Nsim):
        arribos, servicios, esperas, uso, cola = queueSystem()
        tasa = tasa_servicio(uso)
        for i in range(48):
            tasa_est[i] += tasa[i]
            
    for i in range(48):
        tasa_est[i] = tasa_est[i] / Nsim 
    return tasa_est

def promedio_uso(Nsim):
    r = 0
    for _ in range(Nsim):
        arribos, servicios, esperas, uso, cola = queueSystem()
        tasa = tasa_servicio(uso)
        procentaje = uso_total_servicio(tasa)

        r += procentaje
    return r / Nsim

promedio_uso(100)       
            

0.17631403787280372

In [7]:
# TIempo promedio de cliente en sistema
def est_mean(tiempos): ##EN horas es aprox 0.02499.., equivale a 90 seg.
    r = 0
    for t in tiempos:
        r += t
    return r / len(tiempos)

est_mean(servicios)

NameError: name 'servicios' is not defined

In [ ]:
"""
QUEDA PENDIENTE:
    -Distribucion de timepos de espera y Histogramas.
    -Dsitribucion de timepo entre arribos
    -Distribucion de servicios
--> Hay que usar la distribucion empirica para obtenerlas.
"""